In [19]:
import zarr
import xarray as xr
import numpy as np
import time
import math
from geopy.distance import geodesic

In [20]:
st = time.time()
ds = xr.open_zarr(
    'GEBCO_2022_sub_ice_topo.zarr', chunks='auto', 
    group='gebco', decode_cf=False, decode_times=False)

et = time.time()
print('Exe time: ', et-st, 'sec')
ds

Exe time:  0.005158662796020508 sec


<xarray.Dataset>
Dimensions:    (lat: 43200, lon: 86400)
Coordinates:
  * lat        (lat) float64 -90.0 -89.99 -89.99 -89.99 ... 89.99 89.99 90.0
  * lon        (lon) float64 -180.0 -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
Data variables:
    crs        |S1 ...
    elevation  (lat, lon) int16 dask.array<chunksize=(675, 2700), meta=np.ndarray>
Attributes: (12/36)
    Conventions:                     CF-1.6, ACDD-1.3
    comment:                         The data in the GEBCO_2022 Grid should n...
    creator_email:                   gdacc@seabed2030.org
    creator_name:                    GEBCO through the Nippon Foundation-GEBC...
    creator_type:                    International organisation
    creator_url:                     https://www.gebco.net
    ...                              ...
    node_offset:                     1.0
    project:                         Nippon Foundation - GEBCO Seabed2030 Pro...
    references:                      DOI: 10.5285/e0f0bb80-ab44-2739-e053-6c8...
    source:                          The GEBCO_2022 Grid is the latest global...
    summary:                         The GEBCO_2022 Grid is a continuous, glo...
    title:                           The GEBCO_2022 Grid - a continuous terra...

In [21]:
# base=90 for latitude; base=180 for longitude
def gridded_arcsec(x, base=90, arc=3600/15):
    return ((int(x) + base)* arc + math.ceil((x-int(x))*arc))

arc = int(3600/15) #arc
print(gridded_arcsec(0, 180, arc))
print(gridded_arcsec(0, 90, arc))
print(gridded_arcsec(-90, 90, arc))
print(gridded_arcsec(90, 90, arc))
print(gridded_arcsec(-180, 180, arc))
print(gridded_arcsec(180, 180, arc))
print(gridded_arcsec(0.0015, 180, arc))
print(gridded_arcsec(-0.00207, 180, arc))
print(gridded_arcsec(-85.375, 90, arc))
print(gridded_arcsec(48.37918, 90, arc))


43200
21600
0
43200
0
86400
43201
43200
1110
33212


In [25]:
arcsec = 15
arc = int(3600/arcsec) #15 arc-second
basex = 180 #-180 - 180 <==> 0 - 360, half is 180
basey = 90  #-90 - 90 <==> 0 - 180, half is 90
halfxidx = 180 * arc   #in netcdf, longitude length = 86400
halfyidx = 90 * arc    #in netcdf, latitude length = 43200
lon = [122.80154260534586,122.80154260534586] #[123, 123.33, 123.38]
lat = [23.782329893520448,23.769188442795453]
subsetFlag = True
zmode = "line"
format = "row"
if (len(lon) != len(lat)):
    print("Input Error: check longitude, latitude length not equal")
    #return
# May not have exact lon, lat in gridded lon-lat, so need to calculate index  
elif (len(lon) == 1):
    st = time.time()
    lon0 = gridded_arcsec(lon[0], basex, arc)
    lat0 = gridded_arcsec(lat[0], basey, arc)
    mlon0x = ds["lon"][lon0]
    mlat0x = ds["lat"][lat0]
    st1 = ds.sel(lon=mlon0x, lat=mlat0x)
    loc1 = [lon[0], lat[0]]
    xt1 = np.array([st1['elevation'].values])
    et = time.time()
    print('Exe time: ', et-st, 'sec')
    print(xt1)
else:
    st = time.time()
    idx1 = np.empty(shape=[0, 2], dtype=np.int16)
    loc1 = np.empty(shape=[0, 2], dtype=float)
    dis1 = np.array([0.0])
    mlon0 = np.min(lon)
    #mlon1 = np.max(lon)
    mlat0 = np.min(lat)
    #mlat1 = np.max(lat)
    #ds_s1 = ds.sel(lon=slice(mlon0, mlon1), lat=slice(mlat0, mlat1))
    mlonbase = gridded_arcsec(mlon0, basex, arc) if subsetFlag else 0 #if do subsetting dataset, reference-0-x,y should be biased
    mlatbase = gridded_arcsec(mlat0, basey, arc) if subsetFlag else 0
    print(mlonbase, mlatbase)
    #mlonidx1 = gridded_arcsec(mlon1, basex, arc)
    #mlatidx1 = gridded_arcsec(mlat1, basey, arc)
    for i in range(len(lon)-1):
        lonidx0 = gridded_arcsec(lon[i], basex, arc)
        latidx0 = gridded_arcsec(lat[i], basey, arc)
        lonidx1 = gridded_arcsec(lon[i+1], basex, arc)
        latidx1 = gridded_arcsec(lat[i+1], basey, arc)
        #ds_s1 = ds.sel(lon=slice(lon[i], lon[i+1]), lat=slice(lat[i], lat[i+1]))
        #index from gridded_arcsec will change is reference is ds_s1, not ds
        if (lonidx0 == lonidx1 and latidx0 == latidx1):
            idx1 = np.append(idx1, [[latidx0-mlatbase, lonidx0-mlonbase]], axis=0)
            loc1 = np.append(loc1, [[lon[i], lat[i]]], axis=0)
        elif zmode == 'point':
            idx1 = np.append(
                idx1, [[latidx0-mlatbase, lonidx0-mlonbase]], axis=0)
            loc1 = np.append(loc1, [[lon[i], lat[i]]], axis=0)
            dist = geodesic((lat[i], lon[i]),
                            (lat[i+1], lon[i+1])).km
            dis1 = np.append(dis1, dist, axis=None)
            if i == len(lon)-2:
                idx1 = np.append(
                    idx1, [[latidx1-mlatbase, lonidx1-mlonbase]], axis=0)
                loc1 = np.append(loc1, [[lon[i+1], lat[i+1]]], axis=0)            
        else:        
            if(lonidx0 == lonidx1):
                stepi = -1 if latidx0 > latidx1 else 1
                for k,y in enumerate(range(latidx0, latidx1, stepi)):
                    idx1 = np.append(idx1, [[y-mlatbase, lonidx0-mlonbase]], axis=0)
                    locy0 = y/arc - basey
                    locy0i= int(locy0)
                    doty0i= locy0-locy0i-0.25/arc #a small bias to make sure it's in grid
                    print("yi: ", y, locy0, locy0i,locy0i + doty0i)
                    loc1 = np.append(loc1, [[lon[i], locy0i + doty0i]], axis=0)
                    if k>=1:
                        lk = len(loc1)-1
                        dist = geodesic((loc1[lk-1, 1], loc1[lk-1, 0]),
                                        (loc1[lk, 1], loc1[lk, 0])).km
                        dis1 = np.append(dis1, dist, axis=None)
                        print("dist: ", dist, " at ", loc1[lk-1, 1], " from ", loc1[lk, 1])                    
            elif(latidx0 == latidx1):
                stepi = -1 if lonidx0 > lonidx1 else 1
                for k,x in enumerate(range(lonidx0, lonidx1, stepi)):
                    idx1 = np.append(idx1, [[latidx0-mlatbase, x-mlonbase]], axis=0)
                    locx0 = x/arc - basex
                    locx0i= int(locx0)
                    dotx0i= locx0-locx0i-0.25/arc #a small bias to make sure it's in grid
                    print("xi: ", x, locx0i + dotx0i)
                    loc1 = np.append(loc1, [[locx0i + dotx0i, lat[i]]], axis=0)
                    if k>=1:
                        lk = len(loc1)-1
                        dist = geodesic((loc1[lk-1, 1], loc1[lk-1, 0]),
                                        (loc1[lk, 1], loc1[lk, 0])).km
                        dis1 = np.append(dis1, dist, axis=None)                    
            else:
                m = (lat[i+1]-lat[i])/(lon[i+1]-lon[i])
                b = lat[i] - m * lon[i] #y = mx + b
                print("m, b:", m ,b)
                stepi = -1 if lonidx0 > lonidx1 else 1
                for k, x in enumerate(range(lonidx0, lonidx1, stepi)):
                    if (k==0): #should consider internal node not repeated twice
                        idx1 = np.append(idx1, [[latidx0-mlatbase, lonidx0-mlonbase]], axis=0)
                        loc1 = np.append(loc1, [[lon[i], lat[i]]], axis=0)
                    else:
                        locx0 = x/arc - basex
                        locx0i= int(locx0)
                        dotx0i= locx0-locx0i-0.25/arc #a small bias to make sure it's in grid
                        locx1 = locx0i + dotx0i 
                        locy1 = m * locx1 + b 
                        y = gridded_arcsec(locy1, basey, arc)
                        idx1 = np.append(idx1, [[y-mlatbase, x-mlonbase]], axis=0)
                        loc1 = np.append(loc1, [[locx1, locy1]], axis=0)
                        #print(x, locx0, dotx0i, locx1, locy1, y)
                        lk = len(loc1)-1
                        dist = geodesic((loc1[lk-1, 1], loc1[lk-1, 0]),
                                        (loc1[lk, 1], loc1[lk, 0])).km
                        dis1 = np.append(dis1, dist, axis=None)

            if i == len(lon)-2 and lonidx1 != idx1[len(idx1)-1, 1] and latidx1 != idx1[len(idx1)-1, 0]:
                idx1 = np.append(
                    idx1, [[latidx1-mlatbase, lonidx1-mlonbase]], axis=0)
                loc1 = np.append(loc1, [[lon[i+1], lat[i+1]]], axis=0)
                lk = len(loc1)-1
                dist = geodesic((loc1[lk-1, 1], loc1[lk-1, 0]),
                                (loc1[lk, 1], loc1[lk, 0])).km
                dis1 = np.append(dis1, dist, axis=None)
                    
    et = time.time()
    print('Exe time: ', et-st, 'sec')
    print(idx1)
    print(loc1)
    print(dis1)
    np.savetxt("simu/test_loc.csv", loc1, delimiter=",", fmt='%f') 


72673 27305
yi:  27308 23.78333333333333 23 23.782291666666666
yi:  27307 23.77916666666667 23 23.778125000000003
dist:  0.46147940024563405  at  23.782291666666666  from  23.778125000000003
yi:  27306 23.775000000000006 23 23.77395833333334
dist:  0.4614791512945255  at  23.778125000000003  from  23.77395833333334
Exe time:  0.0030333995819091797 sec
[[3 0]
 [2 0]
 [1 0]
 [0 0]]
[[122.80154261  23.78229167]
 [122.80154261  23.778125  ]
 [122.80154261  23.77395833]
 [122.80154261  23.76918844]]
[0.         0.4614794  0.46147915 0.5282889 ]


In [8]:
st = time.time()
#mlon0 = np.min(lon) #may cause slice offset to mlonbase, an offset +-1
mlon1 = np.max(lon) + 1.5/arc #to make it larger
#mlat0 = np.min(lat) #may cause slice offset to mlatbase, an offset +-1
mlat1 = np.max(lat) + 1.5/arc
mlon0x= ds["lon"][mlonbase]
mlat0x= ds["lat"][mlatbase]
ds_s1 = ds.sel(lon=slice(mlon0x, mlon1), lat=slice(mlat0x, mlat1)) if subsetFlag else ds
print(ds_s1['elevation'].shape)
#mlonidx0 = gridded_arcsec(mlon0, basex, arc)
#mlatidx0 = gridded_arcsec(mlat0, basey, arc)
#mlonidx1 = gridded_arcsec(mlon1, basex, arc)
#mlatidx1 = gridded_arcsec(mlat1, basey, arc)
loct = tuple(idx1.T)
print(loct)
pt1=ds_s1['elevation'].values[loct]
et = time.time()
print('Get index of points, time: ', et-st, 'sec')
print(pt1)
np.savetxt("simu/test_zseg.csv", pt1, delimiter=",", fmt='%f')

(362, 93)
(array([ 80,  79,  78,  77,  76,  75,  74,  73,  72,  71,  70,  69,  68,
        67,  66,  65,  64,  63,  62,  61,  60,  59,  58,  57,  56,  55,
        54,  53,  52,  51,  50,  49,  48,  47,  46,  45,  44,  43,  42,
        41,  40,  39,  38,  37,  36,  35,  34,  33,  32,  31,  30,  29,
        28,  27,  26,  25,  24,  23,  22,  21,  20,  19,  18,  17,  16,
        15,  14,  13,  12,  11,  10,   9,   8,   7,   6,   5,   4,   3,
         2,   1,   0,  47,  77, 107, 137, 167, 197, 227, 257, 287, 317,
       347]), array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91]))
Get index of points, time:  0.014019966125488281 sec
[-4819

In [9]:
ds_s1 = ds.sel(lon=slice(mlon0, mlon1), lat=slice(mlat0, mlat1)) 
print(ds_s1)
print(ds["lat"][26720]) #actually more near 21.34, not 21.33 so slice 21.33 makes the offset 1 possibly!!
print(ds["lon"][72720]) #should use print(ds["lat"][26720]) to calcuate back
print(ds["elevation"].values[26720,72720]) 

<xarray.Dataset>
Dimensions:    (lat: 362, lon: 93)
Coordinates:
  * lat        (lat) float64 21.0 21.01 21.01 21.01 ... 22.49 22.5 22.5 22.51
  * lon        (lon) float64 123.0 123.0 123.0 123.0 ... 123.4 123.4 123.4 123.4
Data variables:
    crs        |S1 ...
    elevation  (lat, lon) int16 dask.array<chunksize=(360, 93), meta=np.ndarray>
Attributes: (12/36)
    Conventions:                     CF-1.6, ACDD-1.3
    comment:                         The data in the GEBCO_2022 Grid should n...
    creator_email:                   gdacc@seabed2030.org
    creator_name:                    GEBCO through the Nippon Foundation-GEBC...
    creator_type:                    International organisation
    creator_url:                     https://www.gebco.net
    ...                              ...
    node_offset:                     1.0
    project:                         Nippon Foundation - GEBCO Seabed2030 Pro...
    references:                      DOI: 10.5285/e0f0bb80-ab44-2739-e053-6c

In [10]:
loc1[:,1]

array([21.33      , 21.326875  , 21.32270833, 21.31854167, 21.314375  ,
       21.31020833, 21.30604167, 21.301875  , 21.29770833, 21.29354167,
       21.289375  , 21.28520833, 21.28104167, 21.276875  , 21.27270833,
       21.26854167, 21.264375  , 21.26020833, 21.25604167, 21.251875  ,
       21.24770833, 21.24354167, 21.239375  , 21.23520833, 21.23104167,
       21.226875  , 21.22270833, 21.21854167, 21.214375  , 21.21020833,
       21.20604167, 21.201875  , 21.19770833, 21.19354167, 21.189375  ,
       21.18520833, 21.18104167, 21.176875  , 21.17270833, 21.16854167,
       21.164375  , 21.16020833, 21.15604167, 21.151875  , 21.14770833,
       21.14354167, 21.139375  , 21.13520833, 21.13104167, 21.126875  ,
       21.12270833, 21.11854167, 21.114375  , 21.11020833, 21.10604167,
       21.101875  , 21.09770833, 21.09354167, 21.089375  , 21.08520833,
       21.08104167, 21.076875  , 21.07270833, 21.06854167, 21.064375  ,
       21.06020833, 21.05604167, 21.051875  , 21.04770833, 21.04

In [15]:
from fastapi.encoders import jsonable_encoder
#from fastapi.responses import JSONResponse
json_data = jsonable_encoder({"longitude": loc1[:,0].tolist(), "latitude": loc1[:,1].tolist(),"z": pt1.tolist()})
#print(json_data)

import pandas as pd

df1 = pd.DataFrame({"longitude": loc1[:,0].tolist(), "latitude": loc1[:,1].tolist(),"z": pt1.tolist()}, columns=['longitude', 'latitude', 'z'])
print(df1)

     longitude   latitude     z
0   123.000000  21.330000 -4819
1   123.003125  21.326875 -4905
2   123.007292  21.322708 -5030
3   123.011458  21.318542 -5056
4   123.015625  21.314375 -5108
..         ...        ...   ...
87  123.361458  21.943750 -5307
88  123.365625  22.068750 -5672
89  123.369792  22.193750 -5344
90  123.373958  22.318750 -5288
91  123.378125  22.443750 -4940

[92 rows x 3 columns]


In [12]:
from fastapi.encoders import jsonable_encoder
#just some  simple testing for query string
lax = "25.5, 30.22,20"
lox = " 125.33"
#la1 = lax.split(",")
if (',' in lax):
    try:
        la1 = [float(x.strip()) for x in lax.split(',')]
    except ValueError:
        print("Check your input format") 
else: 
    la1 = [float(lax.strip())]

if (',' in lox):
    lo1 = lox.split(",")
else: 
    lo1 = [float(lox.strip())]
print(la1)
print(lo1)

lon0 = gridded_arcsec(123, basex, arc)
lat0 = gridded_arcsec(21.33, basey, arc)
mlon0x = ds["lon"][lon0]
mlat0x = ds["lat"][lat0]
st1 = ds.sel(lon=mlon0x, lat=mlat0x)
xt1 = np.array([st1['elevation'].values])
print(xt1)
jt1 = {"data": xt1.tolist()}
jsonable_encoder(jt1)

[25.5, 30.22, 20.0]
[125.33]
[-4819]


{'data': [-4819]}

In [13]:
dis1 = np.array([0])
st = time.time()
for i in range(len(loc1)-1):
  dist = geodesic((loc1[i,1],loc1[i,0]), (loc1[i+1,1],loc1[i+1,0])).km
  dis1 = np.append(dis1, dist, axis=None)
et = time.time()
print('Calculate loop of geo dist time: ', et-st, 'sec')
print(dis1)

Calculate loop of geo dist time:  0.014241695404052734 sec
[ 0.          0.47415076  0.63220816  0.63221634  0.63222451  0.63223268
  0.63224085  0.63224902  0.63225719  0.63226536  0.63227352  0.63228168
  0.63228985  0.63229801  0.63230617  0.63231432  0.63232248  0.63233063
  0.63233879  0.63234694  0.63235509  0.63236324  0.63237139  0.63237953
  0.63238768  0.63239582  0.63240396  0.63241211  0.63242024  0.63242838
  0.63243652  0.63244465  0.63245279  0.63246092  0.63246905  0.63247718
  0.63248531  0.63249344  0.63250156  0.63250969  0.63251781  0.63252593
  0.63253405  0.63254217  0.63255028  0.6325584   0.63256651  0.63257463
  0.63258274  0.63259085  0.63259896  0.63260706  0.63261517  0.63262327
  0.63263138  0.63263948  0.63264758  0.63265568  0.63266377  0.63267187
  0.63267996  0.63268806  0.63269615  0.63270424  0.63271233  0.63272042
  0.6327285   0.63273659  0.63274467  0.63275275  0.63276083  0.63276891
  0.63277699  0.63278506  0.63279314  0.63280121  0.63280928  0.6

In [18]:
#https://stackoverflow.com/questions/72620476/faster-way-to-calculate-distance-from-coordinates-on-dataframe
st = time.time()
df2 = pd.merge(df1.rename(columns={'latitude':'clat','longitude':'clon'}), df1, how='cross') \
            .assign(distance=lambda r: r.apply(lambda x: geodesic((x['clat'],x['clon']),(x['latitude'],x['longitude'])).km, axis=1)
            ) #.drop(columns=['clat','clon'])
et = time.time()
print('Calculate df of geo dist time: ', et-st, 'sec')
print(df2)

Calculate df of geo dist time:  1.2061617374420166 sec
            clon      clat   z_x   longitude   latitude   z_y   distance
0     123.000000  21.33000 -4819  123.000000  21.330000 -4819   0.000000
1     123.000000  21.33000 -4819  123.003125  21.326875 -4905   0.474151
2     123.000000  21.33000 -4819  123.007292  21.322708 -5030   1.106359
3     123.000000  21.33000 -4819  123.011458  21.318542 -5056   1.738575
4     123.000000  21.33000 -4819  123.015625  21.314375 -5108   2.370800
...          ...       ...   ...         ...        ...   ...        ...
8459  123.378125  22.44375 -4940  123.361458  21.943750 -5307  55.393120
8460  123.378125  22.44375 -4940  123.365625  22.068750 -5672  41.545140
8461  123.378125  22.44375 -4940  123.369792  22.193750 -5344  27.696961
8462  123.378125  22.44375 -4940  123.373958  22.318750 -5288  13.848581
8463  123.378125  22.44375 -4940  123.378125  22.443750 -4940   0.000000

[8464 rows x 7 columns]
